<a href="https://colab.research.google.com/github/DmitryKutsev/ml_hw/blob/master/hw4_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/DmitryKutsev/ml_hw/master/small_data.csv


--2020-03-24 11:14:17--  https://raw.githubusercontent.com/DmitryKutsev/ml_hw/master/small_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24394136 (23M) [text/plain]
Saving to: ‘small_data.csv’

small_data.csv      100%[===================>]  23.26M  40.8MB/s    in 0.6s    

2020-03-24 11:14:17 (40.8 MB/s) - ‘small_data.csv’ saved [24394136/24394136]



In [0]:
import pandas as pd
from sklearn import cluster
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AffinityPropagation, MeanShift, AgglomerativeClustering
from sklearn.decomposition import TruncatedSVD, NMF

In [0]:

from sklearn.cluster import AffinityPropagation, AgglomerativeClustering, DBSCAN, \
                            KMeans, MiniBatchKMeans, Birch, MeanShift, SpectralClustering
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, \
                            silhouette_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn import metrics
from sklearn.datasets import make_blobs

In [0]:
data = pd.read_csv('small_data.csv')

In [5]:
data.head(5)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1
0,793dd3d5f0f0,9467e0341c0e,Алтайский край,Барнаул,Личные вещи,Детская одежда и обувь,Для девочек,Платья и юбки,146-152 см (10-12 лет),Платье праздничное для девочки,Очень красивое торг,1500.0,106,2017-04-12,Company,ace7b2b7f066a6b6ff244f90d030c2c1a1bf118dd0e1ac...,573.0
1,f8c965d9b668,1905516660f5,Свердловская область,Каменск-Уральский,Для дома и дачи,Мебель и интерьер,"Кровати, диваны и кресла",NaN,NaN,Кровать двухспальная,Двуспальная кровать 140х200 с металлическим ка...,5700.0,10,2017-04-15,Private,b38ef24dd5b876ed626bd98241e228e4e19e4160d33abe...,1439.0
2,2f153c00d561,b4dc4849fdd4,Самарская область,Самара,Бытовая электроника,Телефоны,Samsung,NaN,NaN,Samsung S3 mini gt-i8190,"Продам телефон самсунг 2 ядра, 1 гиг оперативн...",1500.0,6,2017-04-12,Private,8530cc0c8d5dc6d212497dffd610d89b92a7db952934db...,2932.0
3,7fc5eea56915,02c14d39fdb4,Саратовская область,Саратов,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,44–46 (M),"Красивое платье на одно плечо, в идеале размер м","Заказывала в Сша, прекрасное качество, в деле ...",1000.0,1280,2017-04-13,Company,eb21745ba276d3b00b05856a58fe719e31a82419c0221a...,386.0
4,b01000d29ea3,b17f14110367,Удмуртия,Ижевск,Личные вещи,Детская одежда и обувь,Для мальчиков,"Шапки, варежки, шарфы",NaN,Стильная панама,Состояние новой. Размер до года- полтора. По ...,150.0,145,2017-04-14,Company,5bf1001fa681b96f121ad8047c85e3747f3dac37830dd0...,209.0


In [0]:
data = data[['category_name', 'title']]

In [7]:
data.shape

(37339, 2)

In [0]:
sample = data.sample(frac=0.1)

In [9]:
sample.shape

(3734, 2)

In [0]:
#################################################

In [0]:

cv = CountVectorizer(max_features=500)
X = cv.fit_transform(sample['title'])

y = sample['category_name']

In [12]:
cluster = AffinityPropagation(damping=0.7, preference=-5, 
                              max_iter=400, verbose=2)
cluster.fit(X)

Converged after 195 iterations.


AffinityPropagation(affinity='euclidean', convergence_iter=15, copy=True,
                    damping=0.7, max_iter=400, preference=-5, verbose=2)

In [0]:
labels = cluster.labels_

In [14]:
n_clusters = len(set(labels))
n_clusters

210

In [15]:
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

Silhouette Coefficient: 0.344


In [16]:
# эти три метрки что-то вроде точности, полноты и ф-меры для кластеризации
print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, labels)) # проверяет, что в кластере объекты одного класса
print("Completeness: %0.3f" % metrics.completeness_score(y, labels)) # проверяет, что объекты класса только в одном кластере
print("V-measure: %0.3f" % metrics.v_measure_score(y, labels)) # превращает обе метрики в одну 

print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(y, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(y, labels))

Homogeneity: 0.513
Completeness: 0.384
V-measure: 0.439
Adjusted Rand Index: 0.002
Adjusted Mutual Information: 0.318


In [0]:
sample['cluster'] = cluster.labels_

In [18]:
sample[sample.cluster==19].head(10)

,category_name,title,cluster
35577,"Одежда, обувь, аксессуары",Рубашка,19
31873,"Одежда, обувь, аксессуары",Рубашка love republik,19
20384,"Одежда, обувь, аксессуары",Рубашка,19
34205,Детская одежда и обувь,Рубашка школьная,19
25228,"Одежда, обувь, аксессуары",Белая рубашка savage 42 р-р,19
18599,Детская одежда и обувь,"Рубашка на мальчика, фирменная",19
7567,"Одежда, обувь, аксессуары",Рубашка терракотовая,19
10863,"Одежда, обувь, аксессуары",Рубашка,19
17595,"Одежда, обувь, аксессуары",Рубашка Ostin,19
7225,"Одежда, обувь, аксессуары",Рубашка и джинсы,19


In [19]:
sample[sample.category_name=='Телефоны'].cluster.value_counts()

1      45
178    16
113    12
123     9
100     9
7       8
138     7
173     6
153     6
47      5
4       5
90      5
34      5
2       4
99      4
180     4
149     3
181     3
54      2
70      2
8       2
53      1
130     1
Name: cluster, dtype: int64

In [0]:
########################

In [0]:
def cluster_metrics(cluster, sample, cluster_num, X, y):
  labels = cluster.labels_

  print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X[:10000], labels[:10000]))
  
 
  print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, labels)) 
  print("Completeness: %0.3f" % metrics.completeness_score(y, labels))
  print("V-measure: %0.3f" % metrics.v_measure_score(y, labels)) 

  print("Adjusted Rand Index: %0.3f"
        % metrics.adjusted_rand_score(y, labels))
  print("Adjusted Mutual Information: %0.3f"
        % metrics.adjusted_mutual_info_score(y, labels))
  
  sample['cluster'] = cluster.labels_
  print(sample[sample.cluster==cluster_num].head(7)) 


In [0]:
sample1 = data.sample(frac=0.01)


In [0]:
cv = CountVectorizer(max_features=500)
X_cv = cv.fit_transform(sample1['title'])

y_cv = sample1['category_name']

In [0]:
tfidf = TfidfVectorizer(max_features=500)
X_tf = tfidf.fit_transform(sample1['title'])
y_tf = sample1['category_name']


In [53]:
cluster1_cv = AffinityPropagation(damping=0.7, preference=-5, 
                              max_iter=400, verbose=2)
cluster1_cv.fit(X_cv)
labels1_cv = cluster1_cv.labels_

cluster2_cv = KMeans(n_clusters=40, max_iter=300, n_init=10)
cluster2_cv.fit(X_cv)

Converged after 128 iterations.


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=40, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [59]:
cluster1_tf = AffinityPropagation(damping=0.7, preference=-5, 
                              max_iter=400, verbose=2)

cluster2_tf = KMeans(n_clusters=40, max_iter=300, n_init=10)

cluster2_tf.fit(X_tf)
labels2_tf = cluster2_tf.labels_

cluster1_tf.fit(X_tf)
labels1_tf = cluster1_tf.labels_

Converged after 82 iterations.


In [60]:
cluster_metrics(cluster1_cv, sample1, 11, X_cv, y_cv)

Silhouette Coefficient: 0.078
Homogeneity: 0.289
Completeness: 0.524
V-measure: 0.372
Adjusted Rand Index: 0.029
Adjusted Mutual Information: 0.206
                      category_name  ... cluster
18552  Планшеты и электронные книги  ...      11

[1 rows x 3 columns]


In [61]:
cluster_metrics(cluster2_cv, sample1, 11, X_cv, y_cv)

Silhouette Coefficient: 0.020
Homogeneity: 0.362
Completeness: 0.495
V-measure: 0.418
Adjusted Rand Index: 0.024
Adjusted Mutual Information: 0.212
                  category_name  ... cluster
17018   Музыкальные инструменты  ...      11
11014        Красота и здоровье  ...      11
8235   Оборудование для бизнеса  ...      11
10821         Мебель и интерьер  ...      11
35791             Аудио и видео  ...      11
26850  Оборудование для бизнеса  ...      11
32325           Бытовая техника  ...      11

[7 rows x 3 columns]


In [62]:
cluster_metrics(cluster1_tf, sample1, 11, X_tf, y_tf)

Silhouette Coefficient: 0.021
Homogeneity: 0.309
Completeness: 0.320
V-measure: 0.314
Adjusted Rand Index: 0.071
Adjusted Mutual Information: 0.106
                    category_name  ... cluster
12840     Грузовики и спецтехника  ...      11
17720      Детская одежда и обувь  ...      11
10419                  Автомобили  ...      11
23120   Одежда, обувь, аксессуары  ...      11
19722   Одежда, обувь, аксессуары  ...      11
2850                     Телефоны  ...      11
33052  Товары для детей и игрушки  ...      11

[7 rows x 3 columns]


In [63]:
cluster_metrics(cluster2_tf, sample1, 11, X_tf, y_tf)

Silhouette Coefficient: 0.117
Homogeneity: 0.341
Completeness: 0.497
V-measure: 0.405
Adjusted Rand Index: 0.003
Adjusted Mutual Information: 0.194
      category_name               title  cluster
10419    Автомобили  Daewoo Nexia, 2001       11
22576    Автомобили  Daewoo Matiz, 2007       11


In [0]:
cv = CountVectorizer(max_features=500, ngram_range=(1,2))
X_cv = cv.fit_transform(sample1['title'])

y_cv = sample1['category_name']

In [0]:
tfidf = TfidfVectorizer(max_features=500, ngram_range=(1,2))
X_tf = tfidf.fit_transform(sample1['title'])
y_tf = sample1['category_name']

In [66]:
cluster1_cv = AffinityPropagation(damping=0.7, preference=-5, 
                              max_iter=400, verbose=2)
cluster2_cv = KMeans(n_clusters=40, max_iter=300, n_init=10)

cluster1_cv.fit(X_cv)
labels1_cv = cluster1_cv.labels_
cluster2_cv.fit(X_cv)
labels2_cv = cluster2_cv.labels_

cluster1_tf = AffinityPropagation(damping=0.7, preference=-5, 
                              max_iter=400, verbose=2)
cluster2_tf = KMeans(n_clusters=40, max_iter=300, n_init=10)

cluster1_tf.fit(X_tf)
labels1_tf = cluster1_tf.labels_

cluster2_tf.fit(X_tf)
labels2_tf = cluster2_tf.labels_

Converged after 194 iterations.
Converged after 15 iterations.


In [32]:
cluster_metrics(cluster1_cv, sample1, 11, X_cv, y_cv)

Silhouette Coefficient: 0.091
Homogeneity: 0.352
Completeness: 0.494
V-measure: 0.411
Adjusted Rand Index: 0.013
Adjusted Mutual Information: 0.172
                category_name               title  cluster
21873  Детская одежда и обувь  Куртка весна-осень       11


In [67]:
cluster_metrics(cluster2_cv, sample1, 11, X_cv, y_cv)

Silhouette Coefficient: 0.089
Homogeneity: 0.267
Completeness: 0.498
V-measure: 0.348
Adjusted Rand Index: 0.013
Adjusted Mutual Information: 0.148
              category_name                          title  cluster
25199  Дома, дачи, коттеджи   Дом 100 м² на участке 4 сот.       11
7178   Дома, дачи, коттеджи   Дача 35 м² на участке 8 сот.       11
13780  Дома, дачи, коттеджи   Дом 47 м² на участке 30 сот.       11
407    Дома, дачи, коттеджи    Дом 52 м² на участке 4 сот.       11
6039   Дома, дачи, коттеджи    Дом 64 м² на участке 7 сот.       11
18679  Дома, дачи, коттеджи  Дом 32 м² на участке 7.5 сот.       11


In [68]:
cluster_metrics(cluster1_tf, sample1, 11, X_tf, y_tf)

Silhouette Coefficient: -0.120
Homogeneity: 0.416
Completeness: 0.324
V-measure: 0.364
Adjusted Rand Index: 0.003
Adjusted Mutual Information: 0.013
                    category_name  ... cluster
21169  Товары для детей и игрушки  ...      11
17251  Товары для детей и игрушки  ...      11
11552      Ремонт и строительство  ...      11
28639                  Автомобили  ...      11
34734      Детская одежда и обувь  ...      11
8707    Одежда, обувь, аксессуары  ...      11

[6 rows x 3 columns]


In [69]:
cluster_metrics(cluster2_tf, sample1, 11, X_tf, y_tf)

Silhouette Coefficient: 0.195
Homogeneity: 0.318
Completeness: 0.481
V-measure: 0.383
Adjusted Rand Index: -0.003
Adjusted Mutual Information: 0.161
                   category_name                            title  cluster
32603  Одежда, обувь, аксессуары                   Сапоги женские       11
7111   Одежда, обувь, аксессуары  Сланцы женские Reebok easy tone       11
21904  Одежда, обувь, аксессуары      Ботинки, полусапоги женские       11


In [0]:
#вроде лушче, попробую расширить диапазон нграммов

In [70]:
cv = CountVectorizer(max_features=500, ngram_range=(1,3))
X_cv = cv.fit_transform(sample1['title'])

y_cv = sample1['category_name']

tfidf = TfidfVectorizer(max_features=500, ngram_range=(1,3))
X_tf = tfidf.fit_transform(sample1['title'])
y_tf = sample1['category_name']


cluster1_cv = AffinityPropagation(damping=0.7, preference=-5, 
                              max_iter=400, verbose=2)

cluster2_cv = KMeans(n_clusters=40, max_iter=300, n_init=10)

cluster1_cv.fit(X_cv)
labels1_cv = cluster1_cv.labels_

cluster2_cv.fit(X_cv)
labels2_cv = cluster2_cv.labels_

cluster1_tf = AffinityPropagation(damping=0.7, preference=-5, 
                              max_iter=400, verbose=2)
cluster2_tf = KMeans(n_clusters=40, max_iter=300, n_init=10)


cluster1_tf.fit(X_tf)
labels1_tf = cluster1_tf.labels_

cluster2_tf.fit(X_tf)
labels2_tf = cluster2_tf.labels_

Converged after 149 iterations.
Converged after 15 iterations.


In [71]:
cluster_metrics(cluster1_cv, sample1, 11, X_cv, y_cv)

Silhouette Coefficient: 0.161
Homogeneity: 0.316
Completeness: 0.457
V-measure: 0.374
Adjusted Rand Index: 0.003
Adjusted Mutual Information: 0.134
     category_name                                              title  cluster
4309      Ноутбуки  Ноутбук Asus X554LJ-XX787T i3-4005U/4Gb/500Gb/...       11


In [72]:
cluster_metrics(cluster2_cv, sample1, 11, X_cv, y_cv)

Silhouette Coefficient: 0.134
Homogeneity: 0.257
Completeness: 0.480
V-measure: 0.335
Adjusted Rand Index: 0.013
Adjusted Mutual Information: 0.134
      category_name                          title  cluster
31613      Квартиры   1-к квартира, 35 м², 4/5 эт.       11
16277      Квартиры  1-к квартира, 35 м², 8/19 эт.       11


In [73]:
cluster_metrics(cluster1_tf, sample1, 11, X_tf, y_tf)

Silhouette Coefficient: -0.165
Homogeneity: 0.500
Completeness: 0.361
V-measure: 0.420
Adjusted Rand Index: 0.011
Adjusted Mutual Information: 0.035
                category_name                                    title  cluster
3537   Детская одежда и обувь        Спортивные туфли и босоножки р.34       11
34281   Товары для компьютера                        Powerman 300 ватт       11
32325         Бытовая техника  Продам настольную посудомоечную машинку       11
28697       Земельные участки                    Участок 15 сот. (ИЖС)       11
16647       Предложение услуг                      Металло конструкции       11
6481   Детская одежда и обувь                        Платье для девочк       11
17077       Земельные участки               Участок 11.2 га (СНТ, ДНП)       11


In [74]:
cluster_metrics(cluster2_tf, sample1, 11, X_tf, y_tf)

Silhouette Coefficient: 0.233
Homogeneity: 0.355
Completeness: 0.506
V-measure: 0.417
Adjusted Rand Index: 0.033
Adjusted Mutual Information: 0.201
                    category_name                     title  cluster
4330                   Велосипеды            Велосипед кама       11
33160                  Велосипеды  Продам детский велосипед       11
1862                   Велосипеды          Продаю велосипед       11
7068   Товары для детей и игрушки                 Велосипед       11
10161  Товары для детей и игрушки          Велосипед Larsen       11


In [75]:
#вроде лучше, попробую поменять max_feachers
cv = CountVectorizer(max_features=1000, ngram_range=(1,3))
X_cv = cv.fit_transform(sample1['title'])

y_cv = sample1['category_name']

tfidf = TfidfVectorizer(max_features=1000, ngram_range=(1,3))
X_tf = tfidf.fit_transform(sample1['title'])
y_tf = sample1['category_name']


cluster1_cv = AffinityPropagation(damping=0.7, preference=-5, 
                              max_iter=400, verbose=2)

cluster2_cv = KMeans(n_clusters=40, max_iter=300, n_init=10)

cluster1_cv.fit(X_cv)
labels1_cv = cluster1_cv.labels_

cluster2_cv.fit(X_cv)
labels2_cv = cluster2_cv.labels_

cluster1_tf = AffinityPropagation(damping=0.7, preference=-5, 
                              max_iter=400, verbose=2)
cluster2_tf = KMeans(n_clusters=40, max_iter=300, n_init=10)


cluster1_tf.fit(X_tf)
labels1_tf = cluster1_tf.labels_

cluster2_tf.fit(X_tf)
labels2_tf = cluster2_tf.labels_

Converged after 149 iterations.
Converged after 15 iterations.


In [76]:
cluster_metrics(cluster1_cv, sample1, 11, X_cv, y_cv)

Silhouette Coefficient: 0.070
Homogeneity: 0.497
Completeness: 0.501
V-measure: 0.499
Adjusted Rand Index: 0.021
Adjusted Mutual Information: 0.150
                    category_name               title  cluster
21169  Товары для детей и игрушки  Стул для кормления       11


In [77]:
cluster_metrics(cluster2_cv, sample1, 11, X_cv, y_cv)

Silhouette Coefficient: 0.016
Homogeneity: 0.255
Completeness: 0.524
V-measure: 0.343
Adjusted Rand Index: 0.026
Adjusted Mutual Information: 0.158
                   category_name  ... cluster
34952                 Автомобили  ...      11
182                     Растения  ...      11
11118  Одежда, обувь, аксессуары  ...      11
5861   Одежда, обувь, аксессуары  ...      11
4891         Товары для животных  ...      11
12582     Ремонт и строительство  ...      11
18546          Мебель и интерьер  ...      11

[7 rows x 3 columns]


In [78]:
cluster_metrics(cluster1_tf, sample1, 11, X_tf, y_tf)

Silhouette Coefficient: -0.049
Homogeneity: 0.312
Completeness: 0.279
V-measure: 0.295
Adjusted Rand Index: 0.002
Adjusted Mutual Information: 0.008
                   category_name  ... cluster
12582     Ремонт и строительство  ...      11
31613                   Квартиры  ...      11
8235    Оборудование для бизнеса  ...      11
19158     Детская одежда и обувь  ...      11
30145  Одежда, обувь, аксессуары  ...      11
33269  Одежда, обувь, аксессуары  ...      11
22580  Одежда, обувь, аксессуары  ...      11

[7 rows x 3 columns]


In [79]:
cluster_metrics(cluster2_tf, sample1, 11, X_tf, y_tf)

Silhouette Coefficient: 0.141
Homogeneity: 0.271
Completeness: 0.506
V-measure: 0.353
Adjusted Rand Index: 0.017
Adjusted Mutual Information: 0.141
                   category_name           title  cluster
1784   Одежда, обувь, аксессуары          Пальто       11
22580  Одежда, обувь, аксессуары  Пуховик/пальто       11
11647     Детская одежда и обувь          Пальто       11


In [80]:
#похоже, это хуже. попробую, наоборот, уменьшить
cv = CountVectorizer(max_features=300, ngram_range=(1,3))
X_cv = cv.fit_transform(sample1['title'])

y_cv = sample1['category_name']

tfidf = TfidfVectorizer(max_features=300, ngram_range=(1,3))
X_tf = tfidf.fit_transform(sample1['title'])
y_tf = sample1['category_name']


cluster1_cv = AffinityPropagation(damping=0.7, preference=-5, 
                              max_iter=400, verbose=2)

cluster2_cv = KMeans(n_clusters=40, max_iter=300, n_init=10)

cluster1_cv.fit(X_cv)
labels1_cv = cluster1_cv.labels_

cluster2_cv.fit(X_cv)
labels2_cv = cluster2_cv.labels_

cluster1_tf = AffinityPropagation(damping=0.7, preference=-5, 
                              max_iter=400, verbose=2)
cluster2_tf = KMeans(n_clusters=40, max_iter=300, n_init=10)


cluster1_tf.fit(X_tf)
labels1_tf = cluster1_tf.labels_

cluster2_tf.fit(X_tf)
labels2_tf = cluster2_tf.labels_

Converged after 140 iterations.
Converged after 15 iterations.


In [81]:
cluster_metrics(cluster1_cv, sample1, 11, X_cv, y_cv)

Silhouette Coefficient: 0.236
Homogeneity: 0.283
Completeness: 0.445
V-measure: 0.346
Adjusted Rand Index: -0.003
Adjusted Mutual Information: 0.133
      category_name                         title  cluster
20179      Квартиры  3-к квартира, 56 м², 1/4 эт.       11


In [82]:
cluster_metrics(cluster2_cv, sample1, 11, X_cv, y_cv)

Silhouette Coefficient: 0.225
Homogeneity: 0.298
Completeness: 0.459
V-measure: 0.362
Adjusted Rand Index: 0.003
Adjusted Mutual Information: 0.142
      category_name                             title  cluster
15053      Квартиры      3-к квартира, 54 м², 1/1 эт.       11
13116      Квартиры     1-к квартира, 54 м², 6/10 эт.       11
21054      Квартиры  1-к квартира, 54.6 м², 13/14 эт.       11


In [83]:
cluster_metrics(cluster1_tf, sample1, 11, X_tf, y_tf)

Silhouette Coefficient: -0.362
Homogeneity: 0.558
Completeness: 0.384
V-measure: 0.455
Adjusted Rand Index: 0.012
Adjusted Mutual Information: 0.044
                   category_name  ... cluster
5861   Одежда, обувь, аксессуары  ...      11
32603  Одежда, обувь, аксессуары  ...      11
22599                   Растения  ...      11
36990     Детская одежда и обувь  ...      11
19205          Предложение услуг  ...      11
2272               Спорт и отдых  ...      11
36900  Одежда, обувь, аксессуары  ...      11

[7 rows x 3 columns]


In [84]:
cluster_metrics(cluster2_tf, sample1, 11, X_tf, y_tf)

Silhouette Coefficient: 0.334
Homogeneity: 0.376
Completeness: 0.488
V-measure: 0.425
Adjusted Rand Index: 0.009
Adjusted Mutual Information: 0.199
              category_name                          title  cluster
25199  Дома, дачи, коттеджи   Дом 100 м² на участке 4 сот.       11
7178   Дома, дачи, коттеджи   Дача 35 м² на участке 8 сот.       11
24109  Дома, дачи, коттеджи  Дом 80 м² на участке 5.5 сот.       11
13780  Дома, дачи, коттеджи   Дом 47 м² на участке 30 сот.       11
407    Дома, дачи, коттеджи    Дом 52 м² на участке 4 сот.       11
6039   Дома, дачи, коттеджи    Дом 64 м² на участке 7 сот.       11
18679  Дома, дачи, коттеджи  Дом 32 м² на участке 7.5 сот.       11


In [85]:
# попробую еще уменьшить
cv = CountVectorizer(max_features=100, ngram_range=(1,3))
X_cv = cv.fit_transform(sample1['title'])

y_cv = sample1['category_name']

tfidf = TfidfVectorizer(max_features=50, ngram_range=(1,3))
X_tf = tfidf.fit_transform(sample1['title'])
y_tf = sample1['category_name']


cluster1_cv = AffinityPropagation(damping=0.7, preference=-5, 
                              max_iter=400, verbose=2)

cluster2_cv = KMeans(n_clusters=40, max_iter=300, n_init=10)

cluster1_cv.fit(X_cv)
labels1_cv = cluster1_cv.labels_

cluster2_cv.fit(X_cv)
labels2_cv = cluster2_cv.labels_

cluster1_tf = AffinityPropagation(damping=0.7, preference=-5, 
                              max_iter=400, verbose=2)
cluster2_tf = KMeans(n_clusters=40, max_iter=300, n_init=10)


cluster1_tf.fit(X_tf)
labels1_tf = cluster1_tf.labels_

cluster2_tf.fit(X_tf)
labels2_tf = cluster2_tf.labels_

Converged after 113 iterations.
Converged after 132 iterations.


In [86]:
cluster_metrics(cluster1_cv, sample1, 11, X_cv, y_cv)

Silhouette Coefficient: 0.434
Homogeneity: 0.265
Completeness: 0.453
V-measure: 0.334
Adjusted Rand Index: 0.014
Adjusted Mutual Information: 0.164
                    category_name  ... cluster
12840     Грузовики и спецтехника  ...      11
17720      Детская одежда и обувь  ...      11
36990      Детская одежда и обувь  ...      11
8935   Товары для детей и игрушки  ...      11
16398                    Ноутбуки  ...      11
24944      Детская одежда и обувь  ...      11
26405      Детская одежда и обувь  ...      11

[7 rows x 3 columns]


In [87]:
cluster_metrics(cluster2_cv, sample1, 11, X_cv, y_cv)

Silhouette Coefficient: 0.520
Homogeneity: 0.310
Completeness: 0.414
V-measure: 0.355
Adjusted Rand Index: -0.012
Adjusted Mutual Information: 0.114
           category_name                   title  cluster
15710  Земельные участки  Участок 1.6 сот. (ИЖС)       11
27801  Земельные участки    Участок 4 сот. (ИЖС)       11
34301  Земельные участки    Участок 7 сот. (ИЖС)       11


In [88]:
cluster_metrics(cluster1_tf, sample1, 11, X_tf, y_tf)

Silhouette Coefficient: 0.572
Homogeneity: 0.208
Completeness: 0.461
V-measure: 0.286
Adjusted Rand Index: 0.012
Adjusted Mutual Information: 0.162
Empty DataFrame
Columns: [category_name, title, cluster]
Index: []


In [89]:
cluster_metrics(cluster2_tf, sample1, 11, X_tf, y_tf)

Silhouette Coefficient: 0.854
Homogeneity: 0.381
Completeness: 0.453
V-measure: 0.414
Adjusted Rand Index: 0.001
Adjusted Mutual Information: 0.177
                   category_name                  title  cluster
20979  Одежда, обувь, аксессуары   Продам новые слипоны       11
12379  Одежда, обувь, аксессуары   Слипоны Лакост новые       11
480       Детская одежда и обувь  Продам кросовки новые       11
34308     Детская одежда и обувь             Юбки новые       11


In [0]:
#В среднем, похоже, что Count_Vectorizer лучше.

In [0]:
#sample = data.sample(frac=0.05)
y = sample1['category_name']
X = cv.fit_transform(sample1['title'])
#sample1['cluster'] = cluster.labels_


In [0]:
svd = TruncatedSVD(50)
X_svd = svd.fit_transform(X)

nmf = NMF(50)
X_nmf = nmf.fit_transform(X)

In [102]:
cluster1_svd = AffinityPropagation(damping=0.7, preference=-5, 
                              max_iter=400, verbose=2)

cluster2_svd = KMeans(n_clusters=40, max_iter=300, n_init=10)

cluster1_svd.fit(X_svd)
labels1_cv = cluster1_svd.labels_

cluster2_svd.fit(X_svd)
labels2_svd = cluster2_svd.labels_

cluster1_nmf = AffinityPropagation(damping=0.7, preference=-5, 
                              max_iter=400, verbose=2)
cluster2_nmf = KMeans(n_clusters=40, max_iter=300, n_init=10)


cluster1_nmf.fit(X_nmf)
labels1_nmf = cluster1_nmf.labels_

cluster2_nmf.fit(X_nmf)
labels2_nmf = cluster2_nmf.labels_

Converged after 106 iterations.
Converged after 155 iterations.


In [103]:
cluster_metrics(cluster1_svd, sample1, 11, X_svd, y)

Silhouette Coefficient: 0.121
Homogeneity: 0.725
Completeness: 0.483
V-measure: 0.580
Adjusted Rand Index: 0.068
Adjusted Mutual Information: 0.196
                    category_name           title  cluster
9173    Одежда, обувь, аксессуары  Бомбер Bershka       11
32603   Одежда, обувь, аксессуары  Сапоги женские       11
19458  Товары для детей и игрушки         Самокат       11


In [108]:
cluster_metrics(cluster2_svd, sample1, 11, X_svd, y)

Silhouette Coefficient: 0.596
Homogeneity: 0.341
Completeness: 0.419
V-measure: 0.376
Adjusted Rand Index: -0.023
Adjusted Mutual Information: 0.130
      category_name                          title  cluster
29736      Квартиры  1-к квартира, 32 м², 8/10 эт.       11


In [105]:
cluster_metrics(cluster1_nmf, sample1, 11, X_nmf, y)

Silhouette Coefficient: 0.331
Homogeneity: 0.087
Completeness: 0.854
V-measure: 0.158
Adjusted Rand Index: 0.023
Adjusted Mutual Information: 0.122
Empty DataFrame
Columns: [category_name, title, cluster]
Index: []


In [107]:
cluster_metrics(cluster2_nmf, sample1, 11, X_nmf, y)

Silhouette Coefficient: 0.662
Homogeneity: 0.374
Completeness: 0.431
V-measure: 0.401
Adjusted Rand Index: -0.010
Adjusted Mutual Information: 0.154
                   category_name                          title  cluster
33241     Детская одежда и обувь  Платье для выпускного дет сад       11
15278  Одежда, обувь, аксессуары                 Платье женское       11
21091  Одежда, обувь, аксессуары                         Платье       11
24276  Одежда, обувь, аксессуары                         Платье       11
14468     Детская одежда и обувь                         Платье       11
25249  Одежда, обувь, аксессуары    Платье в отличном состоянии       11
6176      Детская одежда и обувь               Выпускное платье       11


In [0]:
#SVD, кажется, получше.

In [0]:
cluster = MeanShift(cluster_all=False, bandwidth=0.5)
cluster.fit(X_svd)
sample1['cluster'] = cluster.labels_
labels = cluster.labels_

In [263]:
sample1.head(3)

,category_name,title,cluster
12840,Грузовики и спецтехника,Продажа Сельхоз-Оборудования Пишите на mail: d...,74
34952,Автомобили,"Nissan Qashqai, 2008",0
17018,Музыкальные инструменты,Продам гитару,2


In [264]:
sample1[sample1.cluster==-1]

,category_name,title,cluster


In [0]:
#пусто, может настройки поменять.

In [0]:
cluster = MeanShift(cluster_all=False, bandwidth=1,min_bin_freq=1)
cluster.fit(X_svd)
sample1['cluster'] = cluster.labels_
labels = cluster.labels_

In [278]:
sample1[sample1.cluster==-1]

,category_name,title,cluster
12840,Грузовики и спецтехника,Продажа Сельхоз-Оборудования Пишите на mail: d...,-1
17720,Детская одежда и обувь,Комбинезон на теплую погоду,-1
31418,"Одежда, обувь, аксессуары",Продам туфли буттери,-1
3537,Детская одежда и обувь,Спортивные туфли и босоножки р.34,-1
5017,Телефоны,USB для iPhone,-1
33241,Детская одежда и обувь,Платье для выпускного дет сад,-1
8235,Оборудование для бизнеса,Продам трансформаторы для прогрева бетона и гр...,-1
32603,"Одежда, обувь, аксессуары",Сапоги женские,-1
16950,Телефоны,Продам Sony z1 copmact (сони компакт зэд1),-1
21573,Детская одежда и обувь,Детские вещи пакетом,-1


In [0]:
 #Ну вот тут, кажется, попадается что-то странное

In [0]:
cluster = MeanShift(cluster_all=False, bandwidth=0.9,min_bin_freq=1)
cluster.fit(X_svd)
sample1['cluster'] = cluster.labels_
labels = cluster.labels_

In [280]:
sample1[sample1.cluster==-1]

,category_name,title,cluster
32603,"Одежда, обувь, аксессуары",Сапоги женские,-1
21573,Детская одежда и обувь,Детские вещи пакетом,-1
35647,"Одежда, обувь, аксессуары","Рубашка в клетку с длинным рукавом, 8XL",-1
10992,Детская одежда и обувь,Сандалики летние 18 размер,-1
22953,Аудио и видео,Радиотехника Усилитель У 101-стерео,-1
16589,Ноутбуки,Asus- Core i7- идеальное состояние как новый,-1
4330,Велосипеды,Велосипед кама,-1
2850,Телефоны,Телефон samsung Galaxy A5 SM-A500F,-1
20114,Аудио и видео,Усилитель sansui G5000,-1
26674,"Одежда, обувь, аксессуары",Кардиган новый,-1


In [0]:
cluster = DBSCAN(min_samples=5, eps=0.5, leaf_size=20) 
cluster.fit(X_svd)
labels = cluster.labels_
sample1['cluster'] = cluster.labels_

In [217]:
sample1[sample1.cluster==-1]

,category_name,title,cluster
12840,Грузовики и спецтехника,Продажа Сельхоз-Оборудования Пишите на mail: d...,-1
32994,Собаки,Девочки сао (алабай) с родословной РКФ,-1
17720,Детская одежда и обувь,Комбинезон на теплую погоду,-1
31613,Квартиры,"1-к квартира, 35 м², 4/5 эт.",-1
31418,"Одежда, обувь, аксессуары",Продам туфли буттери,-1
...,...,...,...
15945,Детская одежда и обувь,Летние туфли,-1
24750,Телефоны,Новый Apple iPhone 7 Plus Silver 128 gb,-1
21904,"Одежда, обувь, аксессуары","Ботинки, полусапоги женские",-1
18331,Земельные участки,Участок 18 сот. (ИЖС),-1


In [0]:
cluster = DBSCAN(min_samples=1.1, eps=0.9, leaf_size=70,  ) 
X = cv.fit_transform(sample1['title'])
X_svd = svd.fit_transform(X)
y = sample['category_name']
cluster.fit(X_svd)
labels = cluster.labels_
sample1['cluster'] = cluster.labels_

In [247]:
sample1[sample1.cluster==-1].head(10)

,category_name,title,cluster
31418,"Одежда, обувь, аксессуары",Продам туфли буттери,-1
5017,Телефоны,USB для iPhone,-1
8235,Оборудование для бизнеса,Продам трансформаторы для прогрева бетона и гр...,-1
26568,Телефоны,Аккумулятор для iPhone 4s,-1
4546,Квартиры,"2-к квартира, 52 м², 4/10 эт.",-1
35791,Аудио и видео,Продам усилитель и две колонки,-1
33210,Детская одежда и обувь,Сандалии для первых шагов малышки,-1
19158,Детская одежда и обувь,Кофта для мальчика,-1
36990,Детская одежда и обувь,"Футболки с длинным рукавом на 2-2,5 года, по 50р",-1
33269,"Одежда, обувь, аксессуары",Новая куртка тонкий синтепон,-1


In [0]:
#Ну вот разве что "Сандалии для первых шагов малышки", но все равно в основном нормальные объявления

In [0]:
cluster = DBSCAN(min_samples=1.1, eps=1.6, leaf_size=70,  ) 
X = cv.fit_transform(sample1['title'])
X_svd = svd.fit_transform(X)
y = sample['category_name']
cluster.fit(X_svd)
labels = cluster.labels_
sample1['cluster'] = cluster.labels_

In [259]:
sample1[sample1.cluster==-1].head(10)

,category_name,title,cluster
13116,Квартиры,"1-к квартира, 54 м², 6/10 эт.",-1
29736,Квартиры,"1-к квартира, 32 м², 8/10 эт.",-1
2281,Телефоны,Чехол на iPhone 4s,-1


In [0]:
#Их все меньше и меньше, похоже трэш найти не выходит.